In [ ]:
# First, we add all the packages we need to the system path.
import os, time
import json
from dotenv import load_dotenv
from contextlib import AsyncExitStack

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import FunctionTool,MessageRole, ListSortOrder, ToolSet
from azure.identity import DefaultAzureCredential

In [ ]:
# Here, we connect to the Azure AI Agents service.
load_dotenv()
project_endpoint = os.getenv("PROJECT_ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")
nonProdTenantId = os.getenv("TENANT_ID")

os.system(f"az config set core.login_experience_v2=off")
os.system(f"az login --tenant {nonProdTenantId}")

subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID_FOR_AGENTS")
print (f"Subscription ID - AZURE_SUBSCRIPTION_ID_FOR_AGENTS : {subscription_id}")

os.system(f"az account set --subscription {subscription_id}")
credential = DefaultAzureCredential()

In [ ]:
# Global variables to maintain state across cells
session = None
exit_stack = None

async def connect_to_server():
    global session, exit_stack

    # Clean up existing connection if any
    if exit_stack:
        await exit_stack.aclose()

    # Create new exit stack
    exit_stack = AsyncExitStack()

    try:
        server_params = StdioServerParameters(
            command="python",
            args=["server.py"],
        )

        # Start the MCP server
        stdio_transport = await exit_stack.enter_async_context(stdio_client(server_params))
        stdio, write = stdio_transport  

        # Create an MCP client session
        session = await exit_stack.enter_async_context(ClientSession(stdio, write))
        await session.initialize()

        # List available tools
        response = await session.list_tools()
        tools = response.tools
        print("\nConnected to server with tools:", [tool.name for tool in tools]) 

        return session, tools
    except Exception as e:
        print(f"Error connecting to server: {e}")
        if exit_stack:
            await exit_stack.aclose()
        raise

# Helper function to clean up resources
async def cleanup_connection():
    global exit_stack
    if exit_stack:
        await exit_stack.aclose()
        exit_stack = None
        print("Connection cleaned up")

In [ ]:
# Connect to MCP server and get tools
session, tools = await connect_to_server()

# Use DefaultAzureCredential for Azure AI Agents
agents_client = AgentsClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(
        exclude_environment_credential=True,
        exclude_managed_identity_credential=True
    )
)

# Build a function for each tool
def make_tool_func(tool_name):
    async def tool_func(**kwargs):
        result = await session.call_tool(tool_name, kwargs)
        return result
    
    tool_func.__name__ = tool_name
    return tool_func

functions_dict = {tool.name: make_tool_func(tool.name) for tool in tools}
mcp_function_tool = FunctionTool(functions=list(functions_dict.values())) 

# Create the agent
agent = agents_client.create_agent(
    model=model_deployment,
    name="inventory-agent",
    instructions="""
    You are an inventory assistant. Here are some general guidelines:
    - Recommend restock if item inventory < 10  and weekly sales > 15
    - Recommend clearance if item inventory > 20 and weekly sales < 5
    Don't ask any follow up qustions, just provide the recommendation.
    """,
    tools=mcp_function_tool.definitions
)

# Enable auto function calling
agents_client.enable_auto_function_calls(tools=mcp_function_tool)   

# Create a thread for the chat session
thread = agents_client.threads.create()

In [ ]:
# Helper function to send a prompt and get response
async def send_prompt(prompt_text, thread_id=None, show_details=False):
    """
    Send a prompt to the agent and return a clean response.
    
    Args:
        prompt_text: The question/prompt to send
        thread_id: Optional thread ID (uses global thread if not provided)
        show_details: Whether to show detailed execution info
    """
    if thread_id is None:
        thread_id = thread.id
        
    print(f"🤖 Sending: {prompt_text}")
    print("─" * 50)
    
    # Create message
    message = agents_client.messages.create(
        thread_id=thread_id,
        role=MessageRole.USER,
        content=prompt_text,
    )
    
    # Create and monitor run
    run = agents_client.runs.create(thread_id=thread_id, agent_id=agent.id)
    
    while run.status in ["queued", "in_progress", "requires_action"]:
        if show_details:
            print(f"Status: {run.status}")
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread_id, run_id=run.id)
        
        if run.status == "requires_action":
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []
            
            for tool_call in tool_calls:
                if show_details:
                    print(f"🔧 Calling tool: {tool_call.function.name}")
                
                # Get function and arguments
                function_name = tool_call.function.name
                args_json = tool_call.function.arguments
                kwargs = json.loads(args_json)
                required_function = functions_dict.get(function_name)
                
                # Execute function
                output = await required_function(**kwargs)
                
                # Append output
                tool_outputs.append({
                    "tool_call_id": tool_call.id,
                    "output": output.content[0].text,
                })
            
            # Submit tool outputs
            agents_client.runs.submit_tool_outputs(
                thread_id=thread_id, 
                run_id=run.id, 
                tool_outputs=tool_outputs
            )
    
    # Handle failures
    if run.status == "failed":
        print(f"❌ Run failed: {run.last_error}")
        return None
    
    # Get the latest response - convert ItemPaged to list first
    messages = agents_client.messages.list(thread_id=thread_id, order=ListSortOrder.DESCENDING)
    messages_list = list(messages)  # Convert ItemPaged to list
    
    if messages_list and messages_list[0].text_messages:
        response = messages_list[0].text_messages[-1].text.value
        print(f"💬 Response:\n{response}")
        print("=" * 50)
        return response
    
    return None

In [ ]:
response = await send_prompt("Can you tell me what the current inventory levels are for all products?")

In [ ]:
response = await send_prompt("Which products would you recommend for clearance?")

In [ ]:
response = await send_prompt("What are the best sellers this week?")

In [ ]:
response = await send_prompt("Which products would you recommend to restock?")

In [ ]:
response = await send_prompt("Can you generate a marketing campaign for products that are under clearance?")